*Note: place API keys in .env*

# Hardcover
- start date: 2025-01-01
- 500 books
- max 20 reviews per book
- min 3 reviews per book

In [3]:
from dotenv import load_dotenv, find_dotenv
import os
import requests
import json
import time
import pandas as pd
from pathlib import Path

load_dotenv(find_dotenv())

API_URL = "https://api.hardcover.app/v1/graphql"
START_DATE = "2025-01-01T00:00:00Z"

# -----------------------------
# SMALL SCOPE SETTINGS (edit these)
# -----------------------------
TARGET_BOOKS = 500        # stop once we collected this many unique books
BOOKS_PAGE_LIMIT = 15      # books per page (keep small for 30s server timeout)
MAX_BOOKS_PAGES = 200      # safety cap
MAX_REVIEWS_PER_BOOK = 20  # cap reviews per book to keep dataset small
MIN_REVIEWS_PER_BOOK = 3   # keep only books with at least this many reviews
SLEEP = 1.1                # <= 60 requests/min safe

# -----------------------------
# Queries
# -----------------------------
BOOKS_PAGE_QUERY = """
query BooksPage($limit: Int!, $offset: Int!) {
  activities(
    limit: $limit
    offset: $offset
    where: { book: { created_at: { _gte: "2025-01-01" } } }
  ) {
    book {
      id
      title
      created_at
      description
      contributions { author { name } }
    }
  }
}
"""


REVIEWS_QUERY = f"""
query ReviewsForBook($book_id: Int!) {{
  user_books(
    where: {{
      book_id: {{ _eq: $book_id }}
      has_review: {{ _eq: true }}
    }}
    order_by: {{ created_at: asc }}
    limit: {MAX_REVIEWS_PER_BOOK}
  ) {{
    created_at
    review_raw
  }}
}}
"""


# -----------------------------
# Helpers
# -----------------------------

def get_token():
    token = os.getenv("HARDCOVER_API_KEY")
    if not token:
        raise RuntimeError("HARDCOVER_API_KEY not found in environment")
    return token[7:] if token.startswith("Bearer ") else token

def graphql_post(token, query, variables, operation_name, retries=6):
    headers = {"Authorization": f"Bearer {token}", "Content-Type": "application/json"}
    payload = {"query": query, "variables": variables, "operationName": operation_name}

    for attempt in range(retries):
        resp = requests.post(API_URL, headers=headers, json=payload, timeout=60)

        if resp.status_code in (408, 429, 500, 502, 503, 504):
            wait = min(2 ** attempt, 30)
            print(f"Transient HTTP {resp.status_code}. Retrying in {wait}s...")
            time.sleep(wait)
            continue

        if resp.status_code != 200:
            raise RuntimeError(f"HTTP {resp.status_code}: {resp.text}")

        data = resp.json()
        if "errors" in data:
            raise RuntimeError(json.dumps(data["errors"], indent=2))

        return data["data"]

    raise RuntimeError("Too many retries")


def extract_authors(book):
    names = []
    for c in (book.get("contributions") or []):
        a = c.get("author")
        if a and a.get("name"):
            names.append(a["name"].strip())
    seen = set()
    out = []
    for n in names:
        if n.lower() not in seen:
            out.append(n)
            seen.add(n.lower())
    return ", ".join(out) if out else None


# -----------------------------
# Step A: Fetch books (offset pagination)
# -----------------------------

def fetch_books_offset(token):
    books_by_id = {}
    offset = 0
    page = 1

    while len(books_by_id) < TARGET_BOOKS and page <= MAX_BOOKS_PAGES:
        print(f"[Books] Page {page} (offset={offset}, limit={BOOKS_PAGE_LIMIT}) | collected={len(books_by_id)}/{TARGET_BOOKS}")

        data = graphql_post(
            token,
            BOOKS_PAGE_QUERY,
            {"limit": BOOKS_PAGE_LIMIT, "offset": offset},
            "BooksPage",
        )

        activities = data.get("activities") or []
        if not activities:
            print("[Books] No more results.")
            break

        for act in activities:
            book = act.get("book") or {}
            book_id = book.get("id")
            if book_id is None:
                continue
            if book_id not in books_by_id:
                books_by_id[book_id] = {
                    "book_id": book_id,
                    "title": book.get("title"),
                    "book_created_at": book.get("created_at"),
                    "description": book.get("description"),
                    "authors": extract_authors(book),
                }

        offset += BOOKS_PAGE_LIMIT
        page += 1
        time.sleep(SLEEP)

    return pd.DataFrame(list(books_by_id.values()))


# -----------------------------
# Step B: Reviews (capped)
# -----------------------------

def fetch_reviews(token, books_df):
    rows = []
    total = len(books_df)

    for i, book_id in enumerate(books_df["book_id"].tolist(), start=1):
        print(f"[Reviews] {i}/{total} book_id={book_id}")
        data = graphql_post(token, REVIEWS_QUERY, {"book_id": int(book_id)}, "ReviewsForBook")

        for r in (data.get("user_books") or []):
            txt = r.get("review_raw")
            if txt and txt.strip():
                rows.append({
                    "book_id": book_id,
                    "review_created_at": r.get("created_at"),
                    "review_raw": txt,
                })

        time.sleep(SLEEP)

    return pd.DataFrame(rows)

# -----------------------------
# Main
# -----------------------------

def main():
    token = get_token()

    print("\n=== STEP A: BOOKS (small scope) ===")
    books_df = fetch_books_offset(token)
    print("Books fetched:", books_df.shape)

    print("\n=== STEP B: REVIEWS (capped) ===")
    reviews_df = fetch_reviews(token, books_df)
    print("Reviews fetched:", reviews_df.shape)

    # Keep only books with >= MIN_REVIEWS_PER_BOOK
    if not reviews_df.empty:
        counts = reviews_df.groupby("book_id").size()
        keep_ids = counts[counts >= MIN_REVIEWS_PER_BOOK].index
        reviews_df = reviews_df[reviews_df["book_id"].isin(keep_ids)].reset_index(drop=True)
        books_df = books_df[books_df["book_id"].isin(keep_ids)].reset_index(drop=True)

    print("\nAfter min-review filter:")
    print("Books:", books_df.shape, "| Reviews:", reviews_df.shape)

    print("\n=== JOIN ===")
    final_df = reviews_df.merge(books_df, on="book_id", how="left")
    print("Final dataset:", final_df.shape)
    print(final_df.head(5).to_string(index=False))

    out = Path('..') / 'data' / "Hardcover_2025.csv"
    (Path('..') / 'data').mkdir(parents=True, exist_ok=True)
    final_df.to_csv(str(out), index=False, encoding="utf-8")
    print(f"\nSaved: {out}")

if __name__ == "__main__":
    main()



=== STEP A: BOOKS (small scope) ===
[Books] Page 1 (offset=0, limit=15) | collected=0/500
[Books] Page 2 (offset=15, limit=15) | collected=14/500
[Books] Page 3 (offset=30, limit=15) | collected=28/500
[Books] Page 4 (offset=45, limit=15) | collected=41/500
[Books] Page 5 (offset=60, limit=15) | collected=55/500
[Books] Page 6 (offset=75, limit=15) | collected=69/500
[Books] Page 7 (offset=90, limit=15) | collected=83/500
[Books] Page 8 (offset=105, limit=15) | collected=95/500
[Books] Page 9 (offset=120, limit=15) | collected=107/500
[Books] Page 10 (offset=135, limit=15) | collected=121/500
[Books] Page 11 (offset=150, limit=15) | collected=136/500
[Books] Page 12 (offset=165, limit=15) | collected=148/500
[Books] Page 13 (offset=180, limit=15) | collected=162/500
[Books] Page 14 (offset=195, limit=15) | collected=173/500
[Books] Page 15 (offset=210, limit=15) | collected=185/500
[Books] Page 16 (offset=225, limit=15) | collected=200/500
[Books] Page 17 (offset=240, limit=15) | coll

# NYT Best Seller

## NYT Books API - Fetch Best Sellers (Author & Title)

In [4]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import os
from pathlib import Path

API_KEY = os.getenv('NYT_API_KEY')

def fetch_bestsellers_for_date(date_str, api_key):
    """Fetch bestsellers using the date-specific endpoint"""
    url = f"https://api.nytimes.com/svc/books/v3/lists/{date_str}/combined-print-and-e-book-fiction.json"
    params = {'api-key': api_key}
    return requests.get(url, params=params)

def get_all_wednesdays(start_date, end_date):
    """Generate all Wednesdays between start and end dates"""
    wednesdays = []
    current = start_date
    
    # Move to first Wednesday (weekday 2)
    while current.weekday() != 2:
        current += timedelta(days=1)
    
    while current <= end_date:
        wednesdays.append(current)
        current += timedelta(days=7)
    
    return wednesdays

# Date range
start_date = datetime(2025, 1, 1)
end_date = datetime.now()

wednesdays = get_all_wednesdays(start_date, end_date)
print(f"Fetching {len(wednesdays)} weeks from {wednesdays[0].date()} to {wednesdays[-1].date()}\n")

all_books = []
successful_weeks = 0
failed_weeks = 0

for i, wednesday in enumerate(wednesdays, 1):
    date_str = wednesday.strftime('%Y-%m-%d')
    
    try:
        response = fetch_bestsellers_for_date(date_str, API_KEY)
        
        if response.status_code == 200:
            data = response.json()
            
            # Navigate to books
            if 'results' in data and 'books' in data['results']:
                books = data['results']['books']
                published_date = data['results'].get('published_date', date_str)
                
                for book in books:
                    all_books.append({
                        'published_date': published_date,
                        'title': book.get('title', 'Unknown'),
                        'author': book.get('author', 'Unknown'),
                        'rank': book.get('rank'),
                        'weeks_on_list': book.get('weeks_on_list', 0)
                    })
                
                successful_weeks += 1
                print(f"✓ [{i}/{len(wednesdays)}] {date_str} → {published_date}: {len(books)} books")
            else:
                failed_weeks += 1
                print(f"⚠ [{i}/{len(wednesdays)}] {date_str}: No books in response")
        
        elif response.status_code == 404:
            failed_weeks += 1
            print(f"⚠ [{i}/{len(wednesdays)}] {date_str}: Not published")
        
        elif response.status_code == 401:
            print(f"✗ [{i}/{len(wednesdays)}] {date_str}: API key error - STOPPING")
            break
        
        else:
            failed_weeks += 1
            print(f"✗ [{i}/{len(wednesdays)}] {date_str}: Error {response.status_code}")
        
        # Rate limit: 5 requests per minute
        time.sleep(12)
        
    except Exception as e:
        failed_weeks += 1
        print(f"✗ [{i}/{len(wednesdays)}] {date_str}: Exception - {e}")

# Save results
print(f"\n{'='*70}")
print(f"Summary: {successful_weeks} successful, {failed_weeks} failed")

if all_books:
    df = pd.DataFrame(all_books)
    
    # Sort by date and rank
    df = df.sort_values(['published_date', 'rank'])
    
    (Path('..') / 'data').mkdir(parents=True, exist_ok=True)
    nyt_path = Path('..') / 'data' / 'nyt_bestsellers_2025.csv'
    df.to_csv(str(nyt_path), index=False)
    
    print(f"✓ Total entries: {len(df)}")
    print(f"✓ Unique books: {df[['title', 'author']].drop_duplicates().shape[0]}")
    print(f"✓ Date range: {df['published_date'].min()} to {df['published_date'].max()}")
    print(f"✓ Saved to: {nyt_path}")
    print(f"{'='*70}")
    
    print("\nSample of data:")
    print(df[['published_date', 'rank', 'title', 'author']].head(10))
    
    print("\nUnique books:")
    unique = df[['title', 'author']].drop_duplicates()
    print(unique.head(10))
    unique_path = Path('..') / 'data' / 'nyt_unique_books_2025.csv'
    unique.to_csv(str(unique_path), index=False)
    print(f"Saved {len(unique)} unique books to {unique_path}")
    
else:
    print("No data collected - check API key and date range")
    print(f"{'='*70}")

Fetching 58 weeks from 2025-01-01 to 2026-02-04

✓ [1/58] 2025-01-01 → 2024-12-29: 15 books
✓ [2/58] 2025-01-08 → 2025-01-05: 15 books
✓ [3/58] 2025-01-15 → 2025-01-12: 15 books
✓ [4/58] 2025-01-22 → 2025-01-19: 15 books
✓ [5/58] 2025-01-29 → 2025-01-26: 15 books
✓ [6/58] 2025-02-05 → 2025-02-02: 15 books
✓ [7/58] 2025-02-12 → 2025-02-09: 15 books
✓ [8/58] 2025-02-19 → 2025-02-16: 15 books
✓ [9/58] 2025-02-26 → 2025-02-23: 15 books
✓ [10/58] 2025-03-05 → 2025-03-02: 15 books
✓ [11/58] 2025-03-12 → 2025-03-09: 15 books
✓ [12/58] 2025-03-19 → 2025-03-16: 15 books
✓ [13/58] 2025-03-26 → 2025-03-23: 15 books
✓ [14/58] 2025-04-02 → 2025-03-30: 15 books
✓ [15/58] 2025-04-09 → 2025-04-06: 15 books
✓ [16/58] 2025-04-16 → 2025-04-13: 15 books
✓ [17/58] 2025-04-23 → 2025-04-20: 15 books
✓ [18/58] 2025-04-30 → 2025-04-27: 15 books
✓ [19/58] 2025-05-07 → 2025-05-04: 15 books
✓ [20/58] 2025-05-14 → 2025-05-11: 15 books
✓ [21/58] 2025-05-21 → 2025-05-18: 15 books
✓ [22/58] 2025-05-28 → 2025-05-25: 1